# Full 3D Model 

In [2]:
import sympy as sp
import numpy as np
from sympy import Matrix, symbols, sin, cos, pi, simplify, pprint, init_printing

# init_printing(use_unicode=True)

### All symbols used / symbolic variables

In [ ]:
# Define symbolic variables
t = sp.Symbol('t', real=True)  # time

# Chassis state variables
x, y, z = sp.symbols('x y z', real=True, cls=sp.Function)
roll, pitch, yaw = sp.symbols('roll pitch yaw', real=True, cls=sp.Function)

# Define functions of time for chassis position and orientation
x = x(t)
y = y(t)
z = z(t)
roll = roll(t)
pitch = pitch(t)
yaw = yaw(t)

# Define derivatives
x_dot = sp.diff(x, t)
y_dot = sp.diff(y, t)
z_dot = sp.diff(z, t)
roll_dot = sp.diff(roll, t)
pitch_dot = sp.diff(pitch, t)
yaw_dot = sp.diff(yaw, t)

# Second derivatives
x_ddot = sp.diff(x_dot, t)
y_ddot = sp.diff(y_dot, t)
z_ddot = sp.diff(z_dot, t)
roll_ddot = sp.diff(roll_dot, t)
pitch_ddot = sp.diff(pitch_dot, t)
yaw_ddot = sp.diff(yaw_dot, t)

# Joint angles for left leg
theta_l1, theta_l2 = sp.symbols('theta_l1 theta_l2', real=True, cls=sp.Function)
theta_l1 = theta_l1(t)
theta_l2 = theta_l2(t)
theta_l1_dot = sp.diff(theta_l1, t)
theta_l2_dot = sp.diff(theta_l2, t)
theta_l1_ddot = sp.diff(theta_l1_dot, t)
theta_l2_ddot = sp.diff(theta_l2_dot, t)

# Joint angles for right leg
theta_r1, theta_r2 = sp.symbols('theta_r1 theta_r2', real=True, cls=sp.Function)
theta_r1 = theta_r1(t)
theta_r2 = theta_r2(t)
theta_r1_dot = sp.diff(theta_r1, t)
theta_r2_dot = sp.diff(theta_r2, t)
theta_r1_ddot = sp.diff(theta_r1_dot, t)
theta_r2_ddot = sp.diff(theta_r2_dot, t)

# Wheel angles
phi_l, phi_r = sp.symbols('phi_l phi_r', real=True, cls=sp.Function)
phi_l = phi_l(t)
phi_r = phi_r(t)
phi_l_dot = sp.diff(phi_l, t)
phi_r_dot = sp.diff(phi_r, t)
phi_l_ddot = sp.diff(phi_l_dot, t)
phi_r_ddot = sp.diff(phi_r_dot, t)

# Control inputs (torques)
tau_l1, tau_l2, tau_lw = sp.symbols('tau_l1 tau_l2 tau_lw', real=True)  # Left leg and wheel torques
tau_r1, tau_r2, tau_rw = sp.symbols('tau_r1 tau_r2 tau_rw', real=True)  # Right leg and wheel torques

# Mass parameters
m_c = sp.Symbol('m_c', positive=True)  # Chassis mass
m_1, m_2 = sp.symbols('m_1 m_2', positive=True)  # Mass of each link
m_w = sp.symbols('m_w', positive=True)  # Wheel masses

# Chassis dimensions
cl_x, cl_y, cl_z = sp.symbols('l_x l_y l_z', positive=True)  # Chassis dimensions

# Link lengths
L_1, L_2 = sp.symbols('L_1 L_2', positive=True)  # Link lengths
r_w = sp.Symbol('r_w', positive=True)  # Wheel radius
W_link = sp.Symbol('W_link', positive=True)  # Link width
H_link = sp.Symbol('H_link', positive=True)  # Link height
H_wheel = sp.Symbol('H_wheel', positive=True)  # Wheel height

# Gravity
g = sp.Symbol('g', positive=True)

# Define the positions of joint attachments on the chassis (in chassis frame)
lx, ly, lz = sp.symbols('lx ly lz', real=True)  # Position offsets of joint from chassis COM

# Inertia tensors
I_c = sp.Matrix(
    [[1/12*m_c*(cl_y**2 + cl_z**2), 0, 0],
     [0, 1/12*m_c*(cl_x**2 + cl_z**2), 0],
     [0, 0, 1/12*m_c*(cl_x**2 + cl_y**2)]]
) # chassis is a block
I_l1 = sp.Matrix(
    [[1/12*m_1*(W_link**2 + H_link**2), 0, 0],
     [0, 1/12*m_1*(L_1**2 + H_link**2), 0],
     [0, 0, 1/12*m_1*(L_1**2 + W_link**2)]]
) # link 1 is a block
I_l2 = sp.Matrix(
    [[1/12*m_2*(W_link**2 + H_link**2), 0, 0],
     [0, 1/12*m_2*(L_2**2 + H_link**2), 0],
     [0, 0, 1/12*m_2*(L_2**2 + W_link**2)]]
) # link 2 is a block
I_w = sp.Matrix(
    [[1/12*m_w*(3*r_w**2 + H_wheel**2), 0, 0],
     [0, 1/2*m_w*r_w**2, 0],
     [0, 0, 1/12*m_w*(3*r_w**2 + H_wheel**2)]]
) # wheel is a cylinder


# Define rotation matrices for chassis orientation
def Rot_x(angle):
    return Matrix([
        [1, 0, 0],
        [0, cos(angle), -sin(angle)],
        [0, sin(angle), cos(angle)]
    ])

def Rot_y(angle):
    return Matrix([
        [cos(angle), 0, sin(angle)],
        [0, 1, 0],
        [-sin(angle), 0, cos(angle)]
    ])

def Rot_z(angle):
    return Matrix([
        [cos(angle), -sin(angle), 0],
        [sin(angle), cos(angle), 0],
        [0, 0, 1]
    ])


### Calculate Positions

In [4]:

# Full rotation matrix (ZYX convention)
R_chassis = Rot_z(yaw) * Rot_y(pitch) * Rot_x(roll)
R_l1 = Rot_y(theta_l1) # Rotation matrix for left leg link 1
R_l2 = Rot_y(theta_l2) # Rotation matrix for left leg link 2
R_r1 = Rot_y(theta_r1) # Rotation matrix for right leg link 1
R_r2 = Rot_y(theta_r2) # Rotation matrix for right leg link 2

# Chassis position in world frame
p_chassis = Matrix([x, y, z])

# Calculate positions of leg joints in world frame
p_left_joint = p_chassis + R_chassis * Matrix([lx, -ly, lz])
p_right_joint = p_chassis + R_chassis * Matrix([lx, ly, lz])

# Calculate positions along the left leg
p_left_link1_end = p_chassis + R_chassis * Matrix([
    lx + -L_1 * cos(theta_l1), 
    -ly, 
    lz + L_1 * sin(theta_l1)])

p_left_link2_end = p_left_link1_end + R_chassis * Matrix([
    -L_2 * cos(theta_l1 + theta_l2),
    0,
    L_2 * sin(theta_l1 + theta_l2)
])

p_left_wheel = p_left_link2_end
# Calculate positions along the right leg
p_right_link1_end = p_right_joint + R_chassis * Matrix([
    lx + L_1 * cos(theta_r1),
    ly,
    lz + L_1 * sin(theta_r1)
])

p_right_link2_end = p_right_link1_end + R_chassis * Matrix([
    L_2 * cos(theta_r1 + theta_r2),
    0,
    L_2 * sin(theta_r1 + theta_r2)
])

p_right_wheel = p_right_link2_end

# Calculate centers of mass (COM) for links (assuming COM at middle of links)
p_left_link1_com = p_left_joint + 0.5 * (p_left_link1_end - p_left_joint)
p_left_link2_com = p_left_link1_end + 0.5 * (p_left_link2_end - p_left_link1_end)
p_right_link1_com = p_right_joint + 0.5 * (p_right_link1_end - p_right_joint)
p_right_link2_com = p_right_link1_end + 0.5 * (p_right_link2_end - p_right_link1_end)


### Velocity and Kinetic Energy Calculations

In [17]:
# Calculate velocities by taking time derivatives
v_chassis = Matrix([x_dot, y_dot, z_dot])

# Calculate angular velocities for each body
omega_chassis = Matrix([roll_dot, pitch_dot, yaw_dot])
omega_left_link1 = omega_chassis + R_chassis * Matrix([0, theta_l1_dot, 0])
omega_left_link2 = omega_left_link1 + R_chassis * R_l1 * Matrix([0, theta_l2_dot, 0])
omega_right_link1 = omega_chassis + R_chassis * Matrix([0, theta_r1_dot, 0])
omega_right_link2 = omega_right_link1 + R_chassis * R_l2 * Matrix([0, theta_r2_dot, 0])
omega_left_wheel = omega_left_link2 + R_chassis * Matrix([0, phi_l_dot, 0])
omega_right_wheel = omega_right_link2 + R_chassis * Matrix([0, phi_r_dot, 0])

# For each body, calculate the velocity of its COM
v_left_link1_com = sp.diff(p_left_link1_com, t)
v_left_link2_com = sp.diff(p_left_link2_com, t)
v_right_link1_com = sp.diff(p_right_link1_com, t)
v_right_link2_com = sp.diff(p_right_link2_com, t)
v_left_wheel = sp.diff(p_left_wheel, t)
v_right_wheel = sp.diff(p_right_wheel, t)

# Calculate kinetic energy terms
# Translational kinetic energy
T_chassis_trans = 0.5 * m_c * (v_chassis.dot(v_chassis))
T_left_link1_trans = 0.5 * m_1 * (v_left_link1_com.dot(v_left_link1_com))
T_left_link2_trans = 0.5 * m_2 * (v_left_link2_com.dot(v_left_link2_com))
T_right_link1_trans = 0.5 * m_1 * (v_right_link1_com.dot(v_right_link1_com))
T_right_link2_trans = 0.5 * m_2 * (v_right_link2_com.dot(v_right_link2_com))
T_left_wheel_trans = 0.5 * m_w * (v_left_wheel.dot(v_left_wheel))
T_right_wheel_trans = 0.5 * m_w * (v_right_wheel.dot(v_right_wheel))

# Rotational kinetic energy (simplified for now, I'll add inertia tensors later)
T_chassis_rot = 0.5 * omega_chassis.dot(R_chassis * I_c * R_chassis.transpose() * omega_chassis)
T_left_link1_rot = 0.5 * omega_left_link1.dot((R_chassis * R_l1) * I_l1 * (R_chassis * R_l1).transpose() * omega_left_link1)
T_left_link2_rot = 0.5 * omega_left_link2.dot((R_chassis * R_l1 * R_l2) * I_l2 * (R_chassis * R_l1 * R_l2).transpose() * omega_left_link2)
T_right_link1_rot = 0.5 * omega_right_link1.dot((R_chassis * R_r1) * I_l1 * (R_chassis * R_r1).transpose() * omega_right_link1)
T_right_link2_rot = 0.5 * omega_right_link2.dot((R_chassis * R_r1 * R_r2) * I_l2 * (R_chassis * R_r1 * R_r2).transpose() * omega_right_link2)
T_left_wheel_rot = 0.5 * omega_left_wheel.dot((R_chassis * R_l1 * R_l2) * I_w * (R_chassis * R_l1 * R_l2).transpose() * omega_left_wheel)
T_right_wheel_rot = 0.5 * omega_right_wheel.dot((R_chassis * R_r1 * R_r2) * I_w * (R_chassis * R_r1 * R_r2).transpose() * omega_right_wheel)

# Total kinetic energy
T = (T_chassis_trans + T_chassis_rot + 
     T_left_link1_trans + T_left_link1_rot + 
     T_left_link2_trans + T_left_link2_rot + 
     T_right_link1_trans + T_right_link1_rot + 
     T_right_link2_trans + T_right_link2_rot + 
     T_left_wheel_trans + T_left_wheel_rot + 
     T_right_wheel_trans + T_right_wheel_rot)


### Potential Energy Calculations

In [18]:

# Potential energy
V_chassis = m_c * g * z
V_left_link1 = m_1 * g * p_left_link1_com[2]
V_left_link2 = m_2 * g * p_left_link2_com[2]
V_right_link1 = m_1 * g * p_right_link1_com[2]
V_right_link2 = m_2 * g * p_right_link2_com[2]
V_left_wheel = m_w * g * p_left_wheel[2]
V_right_wheel = m_w * g * p_right_wheel[2]

# Total potential energy
V = V_chassis + V_left_link1 + V_left_link2 + V_right_link1 + V_right_link2 + V_left_wheel + V_right_wheel


### Non-Holonomic Constraints

In [ ]:
# Wheel Contact Velocities (World Frame)

# Left Wheel Contact Velocity
v_left_contact = v_left_wheel + omega_left_wheel.cross(R_chassis*R_l1*R_l2*Matrix([0, 0, -r_w])) # Velocity of contact point

# Right Wheel Contact Velocity
v_right_contact = v_right_wheel + omega_right_wheel.cross(R_chassis*R_r1*R_r2*Matrix([0, 0, -r_w])) # Velocity of contact point


# Define Wheel Directions (Body Frame)

#  Wheel direction vectors (in the chassis frame) -> point forward along the x-axis of the chassis

u_forwards = R_chassis * Matrix([1, 0, 0]) # Forward direction of chassis
u_perp = R_chassis * Matrix([0,1,0]) # Perpendicular to wheel (no lateral move -> y dir perpendicular)

# Constraint Equations

# No Sideways Slipping (dot product with perpendicular direction = 0)
f1 = v_left_contact.dot(u_perp)   # Left wheel, no lateral slip
f2 = v_right_contact.dot(u_perp)  # Right wheel, no lateral slip

# Rolling Constraint (dot product with wheel direction = r * phi_dot)
f3 = v_left_contact.dot(u_forwards)  # Left wheel rolling no slip (thus v_contact = 0)
f4 = v_right_contact.dot(u_forwards) # Right wheel rolling no lsip

# Lagrange Multipliers
lambda_1, lambda_2, lambda_3, lambda_4 = symbols('lambda_1 lambda_2 lambda_3 lambda_4')


### Lagrangian

In [ ]:
# Lagrangian
L = T - V

# Define generalized coordinates and their derivatives
q = Matrix([x, y, z, roll, pitch, yaw, theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r])
q_dot = Matrix([x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot, 
               theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot])
q_ddot = Matrix([x_ddot, y_ddot, z_ddot, roll_ddot, pitch_ddot, yaw_ddot,
                theta_l1_ddot, theta_l2_ddot, theta_r1_ddot, theta_r2_ddot, phi_l_ddot, phi_r_ddot])

# Define generalized forces
Q = Matrix([0, 0, 0, 0, 0, 0, tau_l1, tau_l2, tau_r1, tau_r2, tau_lw, tau_rw])

#Constraint and Lagrange Multiplier Lists
constraints = [f1, f2, f3, f4]
lambdas = [lambda_1, lambda_2, lambda_3, lambda_4]


### Equations of Motion using Euler-Lagrange equations

In [ ]:

# Euler-Lagrange equations:
# d/dt(∂L/∂q̇) - ∂L/∂q = Q
# -> equations of motion in the form: M(q)q̈ + C(q,q̇)q̇ + G(q) = B·τ
# wheel no slip is non holonomic constraint
def derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas):
    n = len(q)
    m = len(constraints)
    eom = []

    for i in range(n):
        # d/dt(dL/dq_dot)
        term1 = sp.diff(sp.diff(L, q_dot[i]), t)
        # dL/dq
        term2 = sp.diff(L, q[i])

        # Lagrange multiplier term λ (so if f(q, q_dot) is constraint, λ * ∂f/∂qᵢ is added)
        lagrange_term = 0
        for j in range(m):
            lagrange_term += lambdas[j] * sp.diff(constraints[j], q[i])

        # Euler-Lagrange equation (I think it was called Rayleigh's method or D'Alembert's equation in the book, idk I forgor)
        eq = term1 - term2 - Q[i] - lagrange_term
        eom.append(eq)

    # Add the constraint equations
    for constr in constraints:
        eom.append(constr)

    return Matrix(eom)



### Initial Constants and State to Linearize about

In [ ]:
# Initial joint angles
theta_l1_0 = -pi/4
theta_l2_0 = 5 * pi/4
theta_r1_0 = -pi/4
theta_r2_0 = 5 * pi/4

# Initial Chassis Height
z0 = r_w + np.abs(L_1 * sin(theta_l1_0)) + np.abs(L_2 * sin(theta_l2_0))

# Initial state
eq_state = {
    x: 0, y: 0, z: z0,
    roll: 0, pitch: 0, yaw: 0,
    theta_l1: theta_l1_0, theta_l2: theta_l2_0,
    theta_r1: theta_r1_0, theta_r2: theta_r2_0,
    phi_l: 0, phi_r: 0,
    x_dot: 0, y_dot: 0, z_dot: 0,
    roll_dot: 0, pitch_dot: 0, yaw_dot: 0,
    theta_l1_dot: 0, theta_l2_dot: 0,
    theta_r1_dot: 0, theta_r2_dot: 0,
    phi_l_dot: 0, phi_r_dot: 0
}

# Initial control is 0
zero_control_state = {
    tau_l1: 0, tau_l2: 0, tau_lw: 0,
    tau_r1: 0, tau_r2: 0, tau_rw: 0
}

# State vector 
x_state = Matrix([
    x, y, z, roll, pitch, yaw,
    theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r,
    x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot,
    theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot
])

# Control 
u = Matrix([tau_l1, tau_l2, tau_lw, tau_r1, tau_r2, tau_rw])


### Equations of Motion

In [22]:
# eoms = derive_eom(L, q, q_dot, Q)
eoms = derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas)

In [ ]:
# for eq in eoms:
#     pprint(eq)
#     print()

# Write the equations to a latex file
with open('eoms.tex', 'w') as f:
    f.write(sp.latex(eoms))

### Validate EOMS

In [ ]:
# Helper to substitute symbolic "var_dot and var_ddot" for the Derivative(var, t) in the equations
def replace_derivatives(expression, q, q_dot, q_ddot, replace_q, replace_q_dot, replace_q_ddot):
    replacement_dict = {}

    # have to replace in reverse order to avoid matching q_dot to q_ddot

    # Second derivatives
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t, t)] = replace_q_ddot[i]
        #Handle cases of diff(x_dot,t) too
        replacement_dict[sp.Derivative(q_dot[i],t)] = replace_q_ddot[i]
        
    temp_expr = expression.subs(replacement_dict)
    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    replacement_dict = {}

    # First derivatives
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t)] = replace_q_dot[i]
        replacement_dict[q_dot[i]] = replace_q_dot[i]

    temp_expr = temp_expr.subs(replacement_dict)
    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    replacement_dict = {}

    # Generalized coordinates (originaly defined as functions)
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t)] = replace_q[i]

    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    return temp_expr.subs(replacement_dict)
    # return expression.subs(replacement_dict)

# change q variables to a symbol instead of a function
replace_q = [sp.Symbol(f'{str(qi)[:-3]}', real=True) for qi in q]
replace_q_dot = [sp.Symbol(f'{str(qi)}_dot', real=True) for qi in replace_q]
replace_q_ddot = [sp.Symbol(f'{str(qi)}_ddot', real=True) for qi in replace_q]

In [32]:
temp_str = "x(t)"
print(f"{temp_str[:-3]}")

x


In [24]:
# replace Derivative with symbolic representations
eoms_replaced = replace_derivatives(eoms, q, q_dot, q_ddot, replace_q, replace_q_dot, replace_q_ddot)

Derivative(x(t), (t, 2))
Replaced with
x_ddot

Derivative(y(t), (t, 2))
Replaced with
y_ddot

Derivative(z(t), (t, 2))
Replaced with
z_ddot

Derivative(roll(t), (t, 2))
Replaced with
roll_ddot

Derivative(pitch(t), (t, 2))
Replaced with
pitch_ddot

Derivative(yaw(t), (t, 2))
Replaced with
yaw_ddot

Derivative(theta_l1(t), (t, 2))
Replaced with
theta_l1_ddot

Derivative(theta_l2(t), (t, 2))
Replaced with
theta_l2_ddot

Derivative(theta_r1(t), (t, 2))
Replaced with
theta_r1_ddot

Derivative(theta_r2(t), (t, 2))
Replaced with
theta_r2_ddot

Derivative(phi_l(t), (t, 2))
Replaced with
phi_l_ddot

Derivative(phi_r(t), (t, 2))
Replaced with
phi_r_ddot

Derivative(x(t), t)
Replaced with
x_dot

Derivative(y(t), t)
Replaced with
y_dot

Derivative(z(t), t)
Replaced with
z_dot

Derivative(roll(t), t)
Replaced with
roll_dot

Derivative(pitch(t), t)
Replaced with
pitch_dot

Derivative(yaw(t), t)
Replaced with
yaw_dot

Derivative(theta_l1(t), t)
Replaced with
theta_l1_dot

Derivative(theta_l2(t), t)


In [ ]:
solution = sp.solve(eoms, replace_q_ddot + lambdas, dict=True)

if not solution:
    raise ValueError("No symbolic solution found")
if len(solution) > 1:
    print("THere are multiple solutions??")
solution = solution[0]  # take the first solution

# Create expressions for q_ddot in terms of q, q_dot, and torques:
q_ddot_expressions = [solution[q_dd] for q_dd in replace_q_ddot]


In [ ]:
# write solution to file to save (restore later) uise pickle
import pickle as pkl
with open('solution.pkl', 'wb') as f:
    pkl.dump(solution, f)

In [39]:
# Save the replaced eoms to another file
with open('eoms_replaced.tex', 'w') as f:
    f.write(sp.latex(eoms_replaced))

In [25]:
# print(eoms_replaced)

# for every element in oems_replaced, write to a file
with open('eoms_replaced_new.txt', 'w') as f:
    for eq in eoms_replaced:
        f.write(str(eq))
        f.write('\n')